### 1. Importação de bibliotecas e arquivos, inicialização do navegador

In [1]:
# importação de bibliotecas
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import pandas as pd
import time
import win32com.client as win32

# criar um navegador
nav = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# importar/visualizar a base de dados
tabela_produtos = pd.read_excel('buscas.xlsx')
display(tabela_produtos)

,Nome,Termos banidos,Preço mínimo,Preço máximo
0,iphone 12 64 gb,mini watch,3000,3500
1,rtx 3060,zota galax,4000,4500


### 2. Definição das funções de busca no Google e no Buscapé
- busca_google_shopping;
- busca_buscape

In [2]:
def busca_google_shopping(nav, produto, termos_banidos, preco_minimo, preco_maximo):
    
    # tratar os valores da função
    produto = produto.lower()
    termos_banidos = termos_banidos.lower() 
    lista_termos_banidos = termos_banidos.split(' ')
    lista_termos_produto = produto.split(' ')
    preco_maximo = float(preco_maximo)
    preco_minimo = float(preco_minimo)
    
    # entrar no Google
    nav.get('https://www.google.com/')

    # pesquisar o nome do produto no Google
    nav.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(produto) # para o XPATH, é melhor usar aspas simples, pois pode ter aspas duplas no XPATH.
    nav.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

    # clicar na aba "Shopping"
    # pegar a lista de elementos com a mesma classe, percorrer a lista e ver qual tem "Shopping" no text
    elementos = nav.find_elements(By.CLASS_NAME, 'hdtb-mitem')
    for item in elementos:
        if 'Shopping' in item.text:
            item.click()
            break
        
    # pegar o preço, nome e link do produto no Shopping: pegar a lista de resultados da busca no Google Shopping
    lista_resultados = nav.find_elements(By.CLASS_NAME, 'sh-dgr__grid-result')
    # para cada resultado, ele vai verificar se o resultado corresponde a todas as nossas condições
    lista_ofertas = [] # a lista que a função vai me dar como resposta
    for resultado in lista_resultados:
        dic = {}

        nome = resultado.find_element(By.CLASS_NAME, 'Xjkr3b').text
        nome = nome.lower()
        
        # verificação do nome - se no nome tem algum termo banido
        tem_termos_banidos = False
        for palavra in lista_termos_banidos:
            if palavra in nome:
                tem_termos_banidos = True
        # verificar se no nome tem todos os termos do nome do produto
        tem_todos_termos_produto = True
        for palavra in lista_termos_produto:
            if palavra not in nome:
                tem_todos_termos_produto = False

        if not tem_termos_banidos and tem_todos_termos_produto: # verificando o nome
            try:
                preco = resultado.find_element(By.CLASS_NAME, 'a8Pemb').text
                preco = float(preco.replace('R$', '').replace(' ', '').replace('.', '').replace(',', '.'))

                # verificar se algum dos produtos do Google Shopping está dentro da minha faixa de preço (dentro do mínimo e máximo)
                if preco_minimo <= preco <= preco_maximo: # verificando o preço
                    elemento_link = resultado.find_element(By.CLASS_NAME, 'aULzUe')
                    elemento_pai = elemento_link.find_element(By.XPATH, '..')
                    link = elemento_pai.get_attribute('href')
                    lista_ofertas.append((nome, preco, link))
            except:
                continue
    return lista_ofertas

def busca_buscape(nav, produto, termos_banidos, preco_minimo, preco_maximo):
    # tratar os valores da função
    produto = produto.lower()
    preco_minimo = float(preco_minimo)
    preco_maximo = float(preco_maximo)
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(' ')
    lista_termos_produto = produto.split(' ')
    
    # entrar no Buscapé
    nav.get('https://www.buscape.com.br')
    
    # pesquisar pelo produto no Buscapé
    nav.find_element(By.CLASS_NAME, 'AutoCompleteStyle_input__FInnF').send_keys(produto, Keys.ENTER)
    
    # pegar a lista de resultados da busca do Buscapé
    while len(nav.find_elements(By.CLASS_NAME, 'SearchHeader_title__TxP_g')) < 1:
        time.sleep(1)
    time.sleep(5)
    lista_resultados = nav.find_elements(By.CLASS_NAME, 'Cell_Content__fT5st')
    lista_ofertas = []
    # para cada resultado
    for resultado in lista_resultados:
        try:
            nome = resultado.get_attribute('title')
            nome = nome.lower()
            # ver se ele tem algum termo banido
            tem_termos_banidos = False
            for palavra in lista_termos_banidos:
                if palavra in nome:
                    tem_termos_banidos = True
                    
            # ver se ele tem todos os termos do nome do produto
            tem_todos_termos_produto = True
            for palavra in lista_termos_produto:
                if palavra not in nome:
                    tem_todos_termos_produto = False
            
            if not tem_termos_banidos and tem_todos_termos_produto:
                preco = resultado.find_element(By.CLASS_NAME, 'CellPrice_MainValue__JXsj_').text
                preco = float(preco.replace('R$', '').replace(' ', '').replace('.', '').replace(',', '.'))
                # verificar se algum dos produtos do Buscapé está dentro da minha faixa de preço
                # ver se o preço está na faixa de preço ideal
                if preco_minimo <= preco <= preco_maximo:
                    link = resultado.get_attribute('href')
                    lista_ofertas.append((nome, preco, link))
        except:
            pass
    
    # retorna a lista de ofertas
    return lista_ofertas

### 3. Construção da nossa lista de ofertas encontradas

In [3]:
tabela_ofertas = pd.DataFrame()

# para cada item da nossa base de dados (para cada produto)
for linha in tabela_produtos.index:
    produto = tabela_produtos.loc[linha, 'Nome']
    termos_banidos = tabela_produtos.loc[linha, 'Termos banidos']
    preco_minimo = tabela_produtos.loc[linha, 'Preço mínimo']
    preco_maximo = tabela_produtos.loc[linha, 'Preço máximo']

    # procurar esse produto no Google Shopping
    lista_ofertas_google_shopping = busca_google_shopping(nav, produto, termos_banidos, preco_minimo, preco_maximo)
    if lista_ofertas_google_shopping:
        tabela_google_shopping = pd.DataFrame(lista_ofertas_google_shopping, columns = ['produto', 'preco', 'link'])
        tabela_ofertas = tabela_ofertas.append(tabela_google_shopping) # salvar as ofertas boas em um DataFrame (tabela)
    else:
        tabela_google_shopping = None

    # procurar esse produto no Buscapé
    lista_ofertas_buscape = busca_buscape(nav, produto, termos_banidos, preco_minimo, preco_maximo)
    if lista_ofertas_buscape:
        tabela_buscape = pd.DataFrame(lista_ofertas_buscape, columns = ['produto', 'preco', 'link'])
        tabela_ofertas = tabela_ofertas.append(tabela_buscape) # salvar as ofertas boas em um DataFrame (tabela)
    else:
        tabela_buscape = None
    
display(tabela_ofertas)

,produto,preco,link
0,"iphone 11 apple (64gb) branco, tela de 6,1, 4g...",3290.00,https://www.google.com/url?url=https://www.sub...
0,"placa de video asus geforce rtx 3060 ti ko oc,...",4208.96,https://www.google.com/url?url=https://www.sho...
1,placa de vídeo evga nvidia geforce rtx 3060 ti...,4499.90,https://www.google.com/url?url=https://www.sub...
0,placa de video nvidia geforce rtx 3060 ti 8 gb...,4249.00,https://www.buscape.com.br/placa-de-video/plac...
1,placa de video nvidia geforce rtx 3060 12 gb g...,4159.62,https://www.buscape.com.br/placa-de-video/plac...
2,gigabyte placa de vídeo geforce rtx 3060 visio...,4237.51,https://www.buscape.com.br/lead?oid=550418315&...


### 4. Exportar a base de ofertas para Excel

In [4]:
# exportar para o Excel
tabela_ofertas = tabela_ofertas.reset_index(drop=True)
tabela_ofertas.to_excel('Ofertas.xlsx', index=False)

### 5. Enviando o e-mail, e fechando o navegador

In [5]:
# enviar por e-mail o resultado da tabela
if len(tabela_ofertas.index) > 0: # verificando se existe alguma oferta dentro da tabela de ofertas
    outlook = win32.Dispatch('outlook.application')
    email = outlook.CreateItem(0)
    email.To = 'carlos.seiya16@gmail.com'
    email.Subject = 'Produto(s) Encontrado(s) na faixa de preço desejada'
    email.HTMLBody = f'''
    <p>Prezados(as),</p>
    <p>Encontramos alguns produtos em oferta dentro da faixa de preço desejada. Segue tabela com detalhes.
    {tabela_ofertas.to_html(index=False)}
    <p>Qualquer dúvida estou à disposição.</p>
    <p>Att.,</p>
    <p>Carlos</p>
    '''
    email.Send()
    
nav.quit()